<a href="https://colab.research.google.com/github/lili-codelab/comp-linguistics/blob/main/w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [1]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 35.6 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [2]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [3]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [4]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


- **fasttext-wiki-news-subwords-300** обучена на новостных текстах и статьях Википедии с использованием подсловных единиц, за счёт чего хорошо справляется с редкими словами, опечатками и новыми формами. 16B токенов. Часто используется в прикладных задачах: тексты шумнымные или содержат незнакомые слова.

- **glove-wiki-gigaword-300** обучена на текстах Википедии и новостного корпуса Gigaword, ориентирована на более формальный, нейтральный язык. Подходит для задач тематического моделирования, анализа новостных текстов и общих NLP-задач.

- **glove-twitter-100** - предобученные GloVe-векторы, обученные на большом корпусе Twitter (миллионы твитов, миллиарды токенов). Модель отражает особенности разговорной лексики соцсетей: сленг, сокращения, хэштеги, имена собственные и т.п. Полезны для анализа коротких текстов из социальных сетей — для тонального анализа твитов, кластеризации сообщений или классификации по тематике.

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [5]:
import gensim.downloader as api

w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 400000
Размерность векторов: 100


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [9]:
i = input('Введите любое слово ')
close_w = w2v_model.most_similar(i, topn=10)
print(f"Слова, похожие на {i}:")
for word, score in close_w:
    print(f"  {word}: {score:.4f}")

Введите любое слово tea
Слова, похожие на tea:
  coffee: 0.7733
  beer: 0.6795
  drinks: 0.6754
  drink: 0.6684
  lunch: 0.6441
  fruit: 0.6430
  wine: 0.6252
  teas: 0.6108
  cocktail: 0.5971
  juice: 0.5941


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [12]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [15]:
model = Word2Vec(
    sentences = cooking_sentences,
    vector_size = 50,
    window = 3,
    min_count = 1,
    workers = 2,
    sg = 1
)

In [16]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [17]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [18]:
# Найдите слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Найдите слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")


Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [20]:
ft_model = FastText(
    sentences = cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [22]:
try:
    similar = ft_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено")

try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено")

try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено")


Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622
Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944
Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [23]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('мукка')
compare_models('переу')
compare_models('припраа')


Сравнение для слова: 'мукка'
  Word2Vec: слово не найдено
  FastText: ['бекон', 'тост']

Сравнение для слова: 'переу'
  Word2Vec: слово не найдено
  FastText: ['завтрак', 'барбекю']

Сравнение для слова: 'припраа'
  Word2Vec: слово не найдено
  FastText: ['сахар', 'специи']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [24]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [25]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [26]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [27]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [28]:
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [29]:
similar = doc_model.dv.most_similar("doc_1", topn=1)

for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"Самый похожий документ на doc_1:")
    print(f"{doc_tag}: {similarity:.4f}")
    print(f"Текст: {documents[doc_id]}")

Самый похожий документ на doc_1:
doc_1: 0.2735
Текст: deep learning uses neural networks
Самый похожий документ на doc_1:
doc_2: 0.1275
Текст: python programming for data science


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [32]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

test_w = ['мука', 'дрожжи', 'десерт']
v_sizes = [10, 50, 100]

for size in v_sizes:
    ft_model = FastText(
        sentences = cooking_sentences,
        vector_size = size,
        window = 3,
        min_count = 1,
        workers = 2
    )
    print(f"\nРазмерность векторов: {size}")

    for word in test_w:
        try:
            similar_w = ft_model.wv.most_similar(word, topn=5)
            print(f"Слова, похожие на '{word}':")
            for sim_word, score in similar_w:
                print(f"  {sim_word}: {score:.4f}")
        except KeyError:
            print(f"Слово '{word}' не найдено")


Размерность векторов: 10
Слова, похожие на 'мука':
  травы: 0.6755
  варить: 0.6586
  тост: 0.6082
  парить: 0.5879
  духовка: 0.5387
Слова, похожие на 'дрожжи':
  уголь: 0.5751
  соль: 0.5668
  кипятить: 0.5258
  сковорода: 0.5150
  сливки: 0.5005
Слова, похожие на 'десерт':
  сковорода: 0.7641
  паста: 0.6545
  брокколи: 0.5909
  печь: 0.5668
  морковь: 0.4498

Размерность векторов: 50
Слова, похожие на 'мука':
  взбивать: 0.4084
  яйца: 0.3481
  чашка: 0.2984
  начинка: 0.2284
  кипятить: 0.2181
Слова, похожие на 'дрожжи':
  яйца: 0.4264
  готовить: 0.3810
  тесто: 0.2737
  чай: 0.2643
  вино: 0.2585
Слова, похожие на 'десерт':
  молоко: 0.4010
  картофель: 0.3268
  соль: 0.2912
  питание: 0.2701
  яблоки: 0.2586

Размерность векторов: 100
Слова, похожие на 'мука':
  здоровое: 0.2526
  духовка: 0.2187
  соус: 0.2083
  яйца: 0.1555
  травы: 0.1256
Слова, похожие на 'дрожжи':
  уголь: 0.1467
  соль: 0.1397
  помидоры: 0.1350
  картофель: 0.1342
  ингредиенты: 0.1341
Слова, похожие на

10 измерений – слова похожи по числовой близости, но семантика слабая.

50 измерений – оптимально для маленького корпуса, появляются логичные связи.

100 измерений –  слова часто бессмысленно связаны.

Вывод: для маленького корпуса лучше 50 или меньше.